# Naive Bayes - Complete Guide

## 📚 Learning Objectives
- Understand Naive Bayes algorithm and Bayes' theorem
- Implement Gaussian, Multinomial, and Bernoulli Naive Bayes
- Apply to text classification and spam detection
- Understand the "naive" assumption and its implications
- Compare different Naive Bayes variants
- Handle real-world classification problems

## 🎯 What is Naive Bayes?

**Naive Bayes** is a probabilistic classifier based on Bayes' theorem with the "naive" assumption of feature independence.

### Bayes' Theorem:
$$P(y|X) = \frac{P(X|y) \cdot P(y)}{P(X)}$$

Where:
- $P(y|X)$ = Posterior probability (probability of class y given features X)
- $P(X|y)$ = Likelihood (probability of features X given class y)
- $P(y)$ = Prior probability (probability of class y)
- $P(X)$ = Evidence (probability of features X)

### The "Naive" Assumption:
Assumes all features are **independent** given the class:
$$P(X|y) = P(x_1|y) \cdot P(x_2|y) \cdot ... \cdot P(x_n|y)$$

### Types of Naive Bayes:
1. **Gaussian NB**: Continuous features (assumes normal distribution)
2. **Multinomial NB**: Discrete counts (text classification, word counts)
3. **Bernoulli NB**: Binary features (presence/absence)

### When to Use:
✅ Text classification (spam detection, sentiment analysis)  
✅ Document categorization  
✅ Real-time prediction (very fast)  
✅ Small training datasets  
✅ High-dimensional data  
✅ Baseline model  

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)
from sklearn.datasets import load_iris, load_wine, fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

## Part 1: Gaussian Naive Bayes
### For continuous features (assumes normal distribution)
### 1️⃣ Load and Explore Data

In [ ]:
# Load iris dataset
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name='species')

print(f"Dataset shape: {X.shape}")
print(f"\nClasses: {iris.target_names}")
print(f"\nClass distribution:")
print(y.value_counts().sort_index())

# Display statistics
print("\n📊 Feature Statistics:")
print(X.describe())

# Visualize distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, col in enumerate(X.columns):
    for species in range(3):
        data = X[y == species][col]
        axes[idx].hist(data, bins=20, alpha=0.6, label=iris.target_names[species], edgecolor='black')
    
    axes[idx].set_xlabel(col, fontsize=11)
    axes[idx].set_ylabel('Frequency', fontsize=11)
    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Gaussian NB assumes features follow normal distribution within each class")

### 2️⃣ Train Gaussian Naive Bayes

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Predictions
y_pred = gnb.predict(X_test)
y_pred_proba = gnb.predict_proba(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"\n📊 Gaussian Naive Bayes Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"\n📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

# Cross-validation
cv_scores = cross_val_score(gnb, X, y, cv=5, scoring='accuracy')
print(f"\n🔄 Cross-Validation Scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=iris.target_names,
            yticklabels=iris.target_names,
            cbar_kws={'label': 'Count'})
axes[0].set_xlabel('Predicted Label', fontsize=12)
axes[0].set_ylabel('True Label', fontsize=12)
axes[0].set_title('Confusion Matrix - Gaussian NB', fontsize=14, fontweight='bold')

# Prediction probabilities
prob_df = pd.DataFrame(y_pred_proba, columns=iris.target_names)
prob_df['True Class'] = y_test.values
prob_df['Predicted Class'] = y_pred

# Plot probability distribution
for i in range(3):
    class_probs = prob_df[prob_df['True Class'] == i].iloc[:, :3].max(axis=1)
    axes[1].hist(class_probs, bins=20, alpha=0.6, label=iris.target_names[i], edgecolor='black')

axes[1].set_xlabel('Maximum Predicted Probability', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Prediction Confidence Distribution', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3️⃣ Understanding Class Priors and Parameters

In [ ]:
# Class priors (learned from training data)
print("📊 Class Priors (P(y)):")
for i, prior in enumerate(gnb.class_prior_):
    print(f"{iris.target_names[i]}: {prior:.4f}")

# Feature means for each class (theta)
print("\n📊 Feature Means per Class (μ):")
theta_df = pd.DataFrame(gnb.theta_, columns=X.columns, index=iris.target_names)
print(theta_df)

# Feature variances for each class (sigma)
print("\n📊 Feature Variances per Class (σ²):")
sigma_df = pd.DataFrame(gnb.var_, columns=X.columns, index=iris.target_names)
print(sigma_df)

# Visualize means
theta_df.T.plot(kind='bar', figsize=(12, 6), edgecolor='black')
plt.title('Feature Means by Class', fontsize=14, fontweight='bold')
plt.xlabel('Features', fontsize=12)
plt.ylabel('Mean Value', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Species')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## Part 2: Multinomial Naive Bayes
### For discrete count data (text classification)
### 4️⃣ Text Classification Example

In [ ]:
# Load text data (20 newsgroups dataset - subset)
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

print("Loading 20 newsgroups dataset...")
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, 
                                      shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories,
                                     shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))

print(f"\n✅ Data loaded!")
print(f"Training samples: {len(newsgroups_train.data)}")
print(f"Test samples: {len(newsgroups_test.data)}")
print(f"\nCategories: {newsgroups_train.target_names}")

# Show example
print(f"\n📄 Example document:")
print(f"Category: {newsgroups_train.target_names[newsgroups_train.target[0]]}")
print(f"Text: {newsgroups_train.data[0][:300]}...")

In [ ]:
# Create pipeline with CountVectorizer and Multinomial NB
text_clf = Pipeline([
    ('vect', CountVectorizer(max_features=5000, stop_words='english')),
    ('clf', MultinomialNB(alpha=1.0))  # alpha is smoothing parameter
])

# Train
print("Training Multinomial Naive Bayes...")
text_clf.fit(newsgroups_train.data, newsgroups_train.target)
print("✅ Training complete!")

# Predict
y_pred_text = text_clf.predict(newsgroups_test.data)

# Evaluate
accuracy_text = accuracy_score(newsgroups_test.target, y_pred_text)
print(f"\n📊 Multinomial Naive Bayes Performance:")
print(f"Accuracy: {accuracy_text:.4f}")
print(f"\n📋 Classification Report:")
print(classification_report(newsgroups_test.target, y_pred_text, 
                          target_names=newsgroups_test.target_names))

In [ ]:
# Confusion matrix for text classification
cm_text = confusion_matrix(newsgroups_test.target, y_pred_text)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_text, annot=True, fmt='d', cmap='YlOrRd',
            xticklabels=newsgroups_test.target_names,
            yticklabels=newsgroups_test.target_names,
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Category', fontsize=12)
plt.ylabel('True Category', fontsize=12)
plt.title('Confusion Matrix - Text Classification', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

### 5️⃣ Feature Importance in Text Classification

In [ ]:
# Get feature names (words)
feature_names = text_clf.named_steps['vect'].get_feature_names_out()

# Get log probabilities for each class
feature_log_prob = text_clf.named_steps['clf'].feature_log_prob_

# Find top words for each category
n_top_words = 15

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for idx, category in enumerate(newsgroups_test.target_names):
    # Get top features for this category
    top_indices = np.argsort(feature_log_prob[idx])[-n_top_words:]
    top_features = [feature_names[i] for i in top_indices]
    top_scores = feature_log_prob[idx][top_indices]
    
    # Plot
    axes[idx].barh(range(n_top_words), top_scores, color='skyblue', edgecolor='black')
    axes[idx].set_yticks(range(n_top_words))
    axes[idx].set_yticklabels(top_features)
    axes[idx].set_xlabel('Log Probability', fontsize=11)
    axes[idx].set_title(f'Top {n_top_words} Words: {category}', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n💡 These words have highest probability for each category")

### 6️⃣ Smoothing Parameter (Alpha) Tuning

In [ ]:
# Test different alpha values (Laplace smoothing)
alphas = [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
accuracies = []

for alpha in alphas:
    clf = Pipeline([
        ('vect', CountVectorizer(max_features=5000, stop_words='english')),
        ('clf', MultinomialNB(alpha=alpha))
    ])
    
    clf.fit(newsgroups_train.data, newsgroups_train.target)
    y_pred = clf.predict(newsgroups_test.data)
    acc = accuracy_score(newsgroups_test.target, y_pred)
    accuracies.append(acc)
    print(f"Alpha = {alpha:6.3f} -> Accuracy = {acc:.4f}")

# Plot
plt.figure(figsize=(12, 6))
plt.plot(alphas, accuracies, marker='o', linewidth=2, markersize=8, color='green')
plt.xscale('log')
plt.xlabel('Alpha (Smoothing Parameter)', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Impact of Smoothing Parameter on Accuracy', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.axvline(x=alphas[np.argmax(accuracies)], color='red', linestyle='--', 
           label=f'Best Alpha = {alphas[np.argmax(accuracies)]}')
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f"\n🎯 Best Alpha: {alphas[np.argmax(accuracies)]} with Accuracy: {max(accuracies):.4f}")
print("\n💡 Alpha = 1.0 is Laplace smoothing (add-one smoothing)")
print("💡 Smaller alpha = less smoothing (risk of zero probabilities)")
print("💡 Larger alpha = more smoothing (more uniform probabilities)")

## Part 3: Bernoulli Naive Bayes
### For binary features (presence/absence)
### 7️⃣ Binary Feature Classification

In [ ]:
# Create binary features (word presence/absence)
binary_clf = Pipeline([
    ('vect', CountVectorizer(max_features=5000, stop_words='english', binary=True)),
    ('clf', BernoulliNB(alpha=1.0))
])

# Train
print("Training Bernoulli Naive Bayes...")
binary_clf.fit(newsgroups_train.data, newsgroups_train.target)
print("✅ Training complete!")

# Predict
y_pred_binary = binary_clf.predict(newsgroups_test.data)

# Evaluate
accuracy_binary = accuracy_score(newsgroups_test.target, y_pred_binary)
print(f"\n📊 Bernoulli Naive Bayes Performance:")
print(f"Accuracy: {accuracy_binary:.4f}")
print(f"\n📋 Classification Report:")
print(classification_report(newsgroups_test.target, y_pred_binary,
                          target_names=newsgroups_test.target_names))

### 8️⃣ Compare All Three Variants

In [ ]:
# Compare Multinomial vs Bernoulli on text data
# Also compare with TF-IDF

models = [
    ('Multinomial NB (Count)', Pipeline([
        ('vect', CountVectorizer(max_features=5000, stop_words='english')),
        ('clf', MultinomialNB())
    ])),
    ('Multinomial NB (TF-IDF)', Pipeline([
        ('vect', TfidfVectorizer(max_features=5000, stop_words='english')),
        ('clf', MultinomialNB())
    ])),
    ('Bernoulli NB', Pipeline([
        ('vect', CountVectorizer(max_features=5000, stop_words='english', binary=True)),
        ('clf', BernoulliNB())
    ]))
]

results_comparison = []

for name, model in models:
    # Train
    model.fit(newsgroups_train.data, newsgroups_train.target)
    
    # Predict
    y_pred = model.predict(newsgroups_test.data)
    
    # Metrics
    results_comparison.append({
        'Model': name,
        'Accuracy': accuracy_score(newsgroups_test.target, y_pred),
        'Precision': precision_score(newsgroups_test.target, y_pred, average='weighted'),
        'Recall': recall_score(newsgroups_test.target, y_pred, average='weighted'),
        'F1-Score': f1_score(newsgroups_test.target, y_pred, average='weighted')
    })

comparison_df = pd.DataFrame(results_comparison)
print("\n📊 Model Comparison:")
print(comparison_df.to_string(index=False))

# Visualize
comparison_df.set_index('Model').plot(kind='bar', figsize=(14, 6), edgecolor='black')
plt.title('Naive Bayes Variants Comparison', fontsize=14, fontweight='bold')
plt.ylabel('Score', fontsize=12)
plt.xlabel('Model', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Metric', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3, axis='y')
plt.ylim(0.7, 1.0)
plt.tight_layout()
plt.show()

### 9️⃣ Spam Detection Example

In [ ]:
# Create synthetic spam detection dataset
spam_data = [
    # Spam messages
    ("Congratulations! You've won a free iPhone. Click here to claim", 1),
    ("URGENT: Your account will be closed. Verify now", 1),
    ("Get rich quick! Make $5000 per week from home", 1),
    ("Free money! Click here now! Limited time offer!", 1),
    ("You have won the lottery! Send your details to claim prize", 1),
    ("Cheap viagra! Buy now! Special discount!", 1),
    ("Work from home and earn thousands! No experience needed!", 1),
    ("Your package is waiting. Pay shipping fee to receive", 1),
    
    # Ham (legitimate) messages
    ("Hi, are we still meeting for lunch tomorrow?", 0),
    ("The project deadline has been extended to next Friday", 0),
    ("Can you send me the report when you get a chance?", 0),
    ("Thanks for your help with the presentation yesterday", 0),
    ("Meeting scheduled for 3 PM in conference room B", 0),
    ("Please review the attached document and provide feedback", 0),
    ("Your order has been shipped and will arrive in 3-5 days", 0),
    ("Reminder: Team meeting at 10 AM tomorrow", 0)
]

# Expand dataset with variations
spam_messages = [msg for msg, label in spam_data if label == 1] * 10
ham_messages = [msg for msg, label in spam_data if label == 0] * 10

all_messages = spam_messages + ham_messages
all_labels = [1] * len(spam_messages) + [0] * len(ham_messages)

# Split data
X_train_spam, X_test_spam, y_train_spam, y_test_spam = train_test_split(
    all_messages, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

print(f"Spam detection dataset:")
print(f"Training samples: {len(X_train_spam)}")
print(f"Test samples: {len(X_test_spam)}")
print(f"\nClass distribution:")
print(f"Spam: {sum(y_train_spam)} ({sum(y_train_spam)/len(y_train_spam)*100:.1f}%)")
print(f"Ham: {len(y_train_spam) - sum(y_train_spam)} ({(len(y_train_spam) - sum(y_train_spam))/len(y_train_spam)*100:.1f}%)")

# Train spam classifier
spam_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('clf', MultinomialNB(alpha=1.0))
])

spam_clf.fit(X_train_spam, y_train_spam)
y_pred_spam = spam_clf.predict(X_test_spam)

# Evaluate
print(f"\n📊 Spam Detection Performance:")
print(f"Accuracy: {accuracy_score(y_test_spam, y_pred_spam):.4f}")
print(f"\n📋 Classification Report:")
print(classification_report(y_test_spam, y_pred_spam, target_names=['Ham', 'Spam']))

# Test on new messages
test_messages = [
    "Free money! Click now!",
    "Meeting at 3 PM tomorrow",
    "You won a prize! Claim now!",
    "Can you review this document?"
]

predictions = spam_clf.predict(test_messages)
probabilities = spam_clf.predict_proba(test_messages)

print("\n🔍 Test Predictions:")
for msg, pred, prob in zip(test_messages, predictions, probabilities):
    label = 'SPAM' if pred == 1 else 'HAM'
    confidence = prob[pred] * 100
    print(f"\nMessage: '{msg}'")
    print(f"Prediction: {label} (Confidence: {confidence:.1f}%)")

## 📊 Key Takeaways

### Naive Bayes Advantages:
✅ **Fast**: Very quick training and prediction  
✅ **Simple**: Easy to understand and implement  
✅ **Probabilistic**: Provides probability estimates  
✅ **Works with small data**: Effective even with limited training data  
✅ **High-dimensional**: Handles many features well  
✅ **Baseline**: Excellent baseline for text classification  

### Disadvantages:
❌ **Independence assumption**: Assumes features are independent (often violated)  
❌ **Zero probability**: Can assign zero probability to unseen features (use smoothing)  
❌ **Not always accurate**: Simpler models may underperform on complex tasks  

### Which Variant to Use?

| Variant | Use Case | Feature Type | Example |
|---------|----------|--------------|----------|
| **Gaussian NB** | Continuous features | Real numbers | Iris classification, sensor data |
| **Multinomial NB** | Discrete counts | Word counts, frequencies | Text classification, document categorization |
| **Bernoulli NB** | Binary features | Presence/absence | Spam detection, sentiment analysis |

### Best Practices:
1. **Use smoothing** - Always use alpha > 0 to avoid zero probabilities
2. **Feature engineering** - Remove irrelevant features, use domain knowledge
3. **Text preprocessing** - Remove stop words, use stemming/lemmatization
4. **Try different variants** - Test Multinomial vs Bernoulli for text
5. **Tune alpha** - Cross-validate to find optimal smoothing parameter

### When to Use Naive Bayes:
✅ Text classification (spam, sentiment, topic)  
✅ Real-time prediction (very fast)  
✅ Small training datasets  
✅ Baseline model for comparison  
✅ High-dimensional sparse data  
✅ Probabilistic predictions needed  

### When NOT to Use:
❌ Features are highly correlated  
❌ Need maximum accuracy (use ensemble methods)  
❌ Complex feature interactions  
❌ Non-linear decision boundaries  

### Smoothing (Alpha Parameter):
- **Alpha = 0**: No smoothing (risk of zero probabilities)
- **Alpha = 1**: Laplace smoothing (add-one smoothing) - **recommended default**
- **Alpha > 1**: More aggressive smoothing
- **Tune via cross-validation** for best results

### Real-World Applications:
1. **Email spam filtering** - Gmail, Outlook
2. **Sentiment analysis** - Social media monitoring
3. **Document classification** - News categorization
4. **Medical diagnosis** - Disease prediction
5. **Recommendation systems** - Content filtering

### Performance Tips:
- For text: Try both CountVectorizer and TfidfVectorizer
- Limit vocabulary size (max_features) for speed
- Remove stop words and rare words
- Use binary features for presence/absence tasks
- Consider feature selection for high-dimensional data

### Next Steps:
1. Apply to your own text classification problem
2. Compare with Logistic Regression and SVM
3. Try ensemble methods (combine with other classifiers)
4. Experiment with feature engineering
5. Deploy as real-time spam filter